Previously in 2_fullyconnected.ipynb, trained a logistic regression and a neural network model.

The goal of this notebook is to explore regularization techniques.

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

### First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models:

* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### Introduce and tune L2 regularization for both logistic and neural network models.

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # regularization beta
    beta_regul = tf.placeholder(tf.float32)


    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta_regul * tf.nn.l2_loss(weights) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Run it

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.999744
Minibatch accuracy: 7.8%
Validation accuracy: 9.3%
Minibatch loss at step 500: 2.930751
Minibatch accuracy: 74.2%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.836208
Minibatch accuracy: 77.3%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 1.402218
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.892485
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.933271
Minibatch accuracy: 76.6%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.779427
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Test accuracy: 88.0%


### Demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. 

In [7]:
num_steps = 101
num_batches = 10

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.793795
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 10: 8.835071
Minibatch accuracy: 40.6%
Validation accuracy: 44.6%
Minibatch loss at step 20: 6.707327
Minibatch accuracy: 57.8%
Validation accuracy: 56.1%
Minibatch loss at step 30: 6.161949
Minibatch accuracy: 58.6%
Validation accuracy: 61.1%
Minibatch loss at step 40: 5.959385
Minibatch accuracy: 62.5%
Validation accuracy: 64.0%
Minibatch loss at step 50: 5.687385
Minibatch accuracy: 66.4%
Validation accuracy: 66.1%
Minibatch loss at step 60: 5.710994
Minibatch accuracy: 66.4%
Validation accuracy: 67.4%
Minibatch loss at step 70: 5.022999
Minibatch accuracy: 70.3%
Validation accuracy: 68.2%
Minibatch loss at step 80: 5.525428
Minibatch accuracy: 65.6%
Validation accuracy: 68.9%
Minibatch loss at step 90: 4.516294
Minibatch accuracy: 74.2%
Validation accuracy: 70.0%
Minibatch loss at step 100: 5.044063
Minibatch accuracy: 67.2%
Validation accuracy: 70.4%
Test accu

### Introduce Dropout (only training) on the hidden layer of the neural network.

In [8]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)#notice the shape of tf_train_dataset and weights_1
    drop1 = tf.nn.dropout(relu_layer, 0.5)
    logits = tf.matmul(drop1, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [9]:
num_steps = 101
num_batches = 10

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 529.322510
Minibatch accuracy: 5.5%
Validation accuracy: 36.9%
Minibatch loss at step 10: 133.334335
Minibatch accuracy: 71.9%
Validation accuracy: 72.7%
Minibatch loss at step 20: 93.206009
Minibatch accuracy: 74.2%
Validation accuracy: 75.5%
Minibatch loss at step 30: 103.125755
Minibatch accuracy: 69.5%
Validation accuracy: 77.9%
Minibatch loss at step 40: 102.211800
Minibatch accuracy: 77.3%
Validation accuracy: 76.2%
Minibatch loss at step 50: 96.054382
Minibatch accuracy: 71.9%
Validation accuracy: 73.7%
Minibatch loss at step 60: 156.164154
Minibatch accuracy: 67.2%
Validation accuracy: 75.7%
Minibatch loss at step 70: 89.997925
Minibatch accuracy: 71.1%
Validation accuracy: 76.6%
Minibatch loss at step 80: 90.771439
Minibatch accuracy: 79.7%
Validation accuracy: 78.6%
Minibatch loss at step 90: 31.184540
Minibatch accuracy: 76.6%
Validation accuracy: 79.2%
Minibatch loss at step 100: 73.601654
Minibatch accuracy: 73.4%
Validation accuracy: 

### using a multi-layer model to get the best performance

In [19]:
import math as math

batch_size = 128
image_size = 28
num_labels = 10 

beta = 0.05
num_hidden_neurons_1 = 1024
num_hidden_neurons_2 = int(num_hidden_neurons_1 * np.power(0.5, 1)) 
num_hidden_neurons_3 = int(num_hidden_neurons_1 * np.power(0.5, 2))
num_hidden_neurons_4 = int(num_hidden_neurons_1 * np.power(0.5, 3)) 
num_hidden_neurons_5 = int(num_hidden_neurons_1 * np.power(0.5, 4)) 

# create tensorflow graph
graph = tf.Graph()
with graph.as_default():
    # tensorflow graph input
    # At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ## Build four(4) hidden layers with 1024 neurons at each layer.
    ## Hidden layer 1 with ReLUs
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_neurons_1], stddev=math.sqrt(2.0/(image_size * image_size)))) ## stddev normal distribution
    biases_1 = tf.Variable(tf.zeros([num_hidden_neurons_1]))
    
    ## Hidden layer 2 with ReLUs
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_neurons_1, num_hidden_neurons_2], stddev=math.sqrt(2.0/(num_hidden_neurons_1)))) 
    biases_2 = tf.Variable(tf.zeros([num_hidden_neurons_2]))
    
    ## Hidden layer 3 with ReLUs
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden_neurons_2, num_hidden_neurons_3], stddev=math.sqrt(2.0/(num_hidden_neurons_2)))) 
    biases_3 = tf.Variable(tf.zeros([num_hidden_neurons_3]))
    
    ## Hidden layer 4 with ReLUs
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden_neurons_3, num_hidden_neurons_4], stddev=math.sqrt(2.0/(num_hidden_neurons_3)))) 
    biases_4 = tf.Variable(tf.zeros([num_hidden_neurons_4]))
    
    ## Hidden layer 5 with ReLUs
    weights_5 = tf.Variable(tf.truncated_normal([num_hidden_neurons_4, num_hidden_neurons_5], stddev=math.sqrt(2.0/(num_hidden_neurons_4)))) 
    biases_5 = tf.Variable(tf.zeros([num_hidden_neurons_5]))
    
        
    # output layer
    weights_6 = tf.Variable(tf.truncated_normal([num_hidden_neurons_5, num_labels],stddev=math.sqrt(2.0/(num_hidden_neurons_5))))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
                            
    # Training computation.
    # Hidden layer 1 with ReLUs
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 
    relu_layer_1 = tf.nn.relu(logits_1)
    # dropout 
    keep_prob = tf.placeholder(tf.float32) 
    relu_layer_dropout_1 = tf.nn.dropout(relu_layer_1, keep_prob)
    
    ## Hidden layer 2 with ReLUs
    logits_2 = tf.matmul(relu_layer_dropout_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    relu_layer_dropout_2 = tf.nn.dropout(relu_layer_2, keep_prob)
    
    ## Hidden layer 3 with ReLUs
    logits_3 = tf.matmul(relu_layer_dropout_2, weights_3) + biases_3
    relu_layer_3 = tf.nn.relu(logits_3)
    relu_layer_dropout_3 = tf.nn.dropout(relu_layer_3, keep_prob)
    
    ## Hidden layer 4 with ReLUs
    logits_4 = tf.matmul(relu_layer_dropout_3, weights_4) + biases_4
    relu_layer_4 = tf.nn.relu(logits_4)
    relu_layer_dropout_4 = tf.nn.dropout(relu_layer_4, keep_prob)
                            
    ## Hidden layer 5 with ReLUs
    logits_5 = tf.matmul(relu_layer_dropout_4, weights_5) + biases_5
    relu_layer_5 = tf.nn.relu(logits_5)
    relu_layer_dropout_5 = tf.nn.dropout(relu_layer_5, keep_prob)
    
    # output layer
    logits = tf.matmul(relu_layer_dropout_5, weights_6) + biases_6
    
    # loss function + L2 regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) 
    
    # L2 regularization
    regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta*regularization)
    
    # Optimizer.
    ## Use Learning Rate Decaying.
    global_step = tf.Variable(0) ## count the number of steps taken
    starter_learning_rate = 0.5
    number_decay_steps = 100000
    base_of = 0.96

    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, number_decay_steps, base_of, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation 
    logits_valid_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_valid_1 = tf.nn.relu(logits_valid_1)
    # l2
    logits_valid_2 = tf.matmul(relu_layer_valid_1, weights_2) + biases_2
    relu_layer_valid_2 = tf.nn.relu(logits_valid_2)
    # l3
    logits_valid_3 = tf.matmul(relu_layer_valid_2, weights_3) + biases_3
    relu_layer_valid_3 = tf.nn.relu(logits_valid_3)
    # l4
    logits_valid_4 = tf.matmul(relu_layer_valid_3, weights_4) + biases_4
    relu_layer_valid_4 = tf.nn.relu(logits_valid_4)
    # l5
    logits_valid_5 = tf.matmul(relu_layer_valid_4, weights_5) + biases_5
    relu_layer_valid_5 = tf.nn.relu(logits_valid_5)
    
    logits_valid = tf.matmul(relu_layer_valid_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(logits_valid)
    
    # Predictions for test data
    logits_test_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_test_1 = tf.nn.relu(logits_test_1)
    # l2
    logits_test_2 = tf.matmul(relu_layer_test_1, weights_2) + biases_2
    relu_layer_test_2 = tf.nn.relu(logits_test_2)
    # l3
    logits_test_3 = tf.matmul(relu_layer_test_2, weights_3) + biases_3
    relu_layer_test_3 = tf.nn.relu(logits_test_3)
    # l4
    logits_test_4 = tf.matmul(relu_layer_test_3, weights_4) + biases_4
    relu_layer_test_4 = tf.nn.relu(logits_test_4)
    # l5
    logits_test_5 = tf.matmul(relu_layer_test_4, weights_5) + biases_5
    relu_layer_test_5 = tf.nn.relu(logits_test_5)
                                     
    logits_test = tf.matmul(relu_layer_test_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(logits_test)

In [25]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 80.837082
Minibatch accuracy: 7.8%
Validation accuracy: 13.4%
Minibatch loss at step 500: 2.312563
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 2.302711
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 2.305659
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 2.307440
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 2.304433
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 2.300880
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3500: 2.304917
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 4000: 2.305696
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 4500: 2.303373
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 5000: 2.302841
Minibatch accuracy: 12.5%
Validation accuracy: 